# Upload a file

For this example, we assume you have already obtained your Evo hub URL, organization ID, workspace ID, and authorization token. Insert them below.

In [ ]:
# See the Evo documentation for help on obtaining these values
evo_hub_url = "<insert Evo host address here>"
org_id = "<insert org ID here>"
workspace_id = "<insert workspace ID here>"
auth_token = "<insert auth token here>"

## Get blob storage URL

In [ ]:
import requests

file_path = "data/example.txt"
url = f"https://{evo_hub_url}/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_path}"

response = requests.put(url, headers={"Authorization": f"Bearer {auth_token}"})
response_json = response.json()

response_json

## Upload the blob data

In [ ]:
file_contents = {f"{response_json['file_id']}": open(file_path, "rb")}
upload_response = requests.put(
    response_json["upload"],
    files=file_contents,
    headers={"x-ms-blob-type": "BlockBlob"},
)
upload_response

## Poll until the file is ready

This example gives an example of looping while we wait for the file upload to complete in the background.

**Tip:** if you're uploading a new version of an existing file, you can compare the `version_id` you receive in the first and last steps of this example to determine whether the new version has finished uploading. 

In [ ]:
import time

url = f"https://{evo_hub_url}/file/v2/orgs/{org_id}/workspaces/{workspace_id}/files/path/{file_path}"
i = 1
limit = 10  # To prevent an infinite loop

while True:
    file = requests.get(url, headers={"Authorization": f"Bearer {auth_token}"})
    result = file.json()
    # Is the file ready yet?
    if result:
        break
    # Have we reached our recursion limit?
    i += 1
    if i > limit:
        break
    # Wait a bit until trying again
    time.sleep(1)

print("File is ready to use.")